In [ ]:
# visualize feature maps output from each block in the vgg model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.models import Model
from matplotlib import pyplot
from numpy import expand_dims
import numpy as np
from itertools import chain
from keras import backend as K
from keras.layers.convolutional import MaxPooling3D
from vvvv import VVVVClient
import keras
import time
import socket

In [2]:
def startCycle(model, ixs, path, cycle):

    outputs = [model.layers[i].output for i in ixs]
    model = Model(inputs = model.inputs, outputs = outputs)

    # load the image with the required shape
    img = load_img((r"C:\Users\adchu\Desktop\0.png"), target_size = (224, 224))

    # convert the image to an array
    img = img_to_array(img)

    # expand dimensions so that it represents a single 'sample'
    img = expand_dims(img, axis=0)

    # prepare the image (e.g. scale pixel values for the vgg)
    img = preprocess_input(img)

    # get feature map for first hidden layer
    feature_maps = model.predict(img)

    toMax = []

    for i in range(len(feature_maps)):
        toMax.append(feature_maps[i][0].mean(2).flatten().tolist())
    toMaxArray = []
    #print(toMaxArray)

    cycle = 0
    print(cycle)

    func = [3, 2, 1, 1, 1]

    for i in range (len(toMax)):
        n = feature_maps[i][0].shape[0]
        pool = keras.layers.MaxPooling2D(pool_size=(func[i], func[i]))
        b = K.constant(feature_maps[i][0].mean(2).reshape(-1, n, n, 1))
        out = K.eval(pool(b))
        out = (out - out.min()) / (out.max() - out.min())
        #print(out.shape)
        toMaxArray += [out.flatten().tolist()]
    return toMaxArray

def mainCycle(model, ixs, path, cycle):
    
    outputs = [model.layers[i].output for i in ixs]
    model = Model(inputs = model.inputs, outputs = outputs)
    
    # load the image with the required shape
    img = load_img(path, target_size=(224, 224))

    # convert the image to an array
    img = img_to_array(img)

    # expand dimensions so that it represents a single 'sample'
    img = expand_dims(img, axis=0)
    # prepare the image (e.g. scale pixel values for the vgg)
    img = preprocess_input(img)

    # get feature map for first hidden layer
    feature_maps = model.predict(img)


    toMax = []

    for i in range(len(feature_maps)):
        toMax.append(feature_maps[i][0].mean(2).flatten().tolist())

    #print(toMax[4])

    toMaxArray = []
    #print(toMaxArray)
    print(cycle)
    
    func = [3, 2, 1, 1, 1]

    for i in range (len(toMax)):
        n = feature_maps[i][0].shape[0]
        pool = keras.layers.MaxPooling2D(pool_size=(func[i], func[i]))
        b = K.constant(feature_maps[i][0].mean(2).reshape(-1, n, n, 1))
        out = K.eval(pool(b))
        out = (out - out.min()) / (out.max() - out.min())
        #print(out.shape)
        toMaxArray += [out.flatten().tolist()]
    return toMaxArray

def setDelayTime(ip, port):
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    buffer_size = 8192
    s.bind((ip, port))
    #print(client, port)
    s.listen(1)
    (client, (ip,port)) = s.accept()
    s.settimeout(1)

    while True:
        data = client.recv(buffer_size)
        delayTime = 0
        dT = 0

        if len(data) == 0:
#                 print(dT)
            s.close()
            break
        else:
            # got a message do something :)
            delayTime = data.decode()
            dT = float(delayTime.replace('\n', '').replace(';', ''))
            print(dT)
            vc = VVVVClient('127.0.0.1', int('16161'))
            vc.send_msg('/fb', str(1))
            time.sleep(.2)
            vc.send_msg('/fb', str(0))
        s.close()
        return dT

def vcClient(ip, port, toMaxArray, dT = 0.5):
    vc = VVVVClient('127.0.0.1', int('12000'))
    for i in range(len(toMaxArray)):
        vc.send_msg('/conv{}'.format(i), toMaxArray[i])
        vc.send_msg('/trig', str(1))
        time.sleep(.2)
        vc.send_msg('/trig', str(0))
    time.sleep(dT)
#     print('sucks')

In [ ]:
cycle = 0
dT = 0.5

# load the model
model = VGG16()

# redefine model to output right after the first hidden layer
ixs = [2, 5, 9, 13, 17]


toMaxArray = startCycle(model, ixs, r"C:\Users\adchu\Desktop\0.png", cycle)
vcClient('127.0.0.1', '12000', toMaxArray, dT)

# mainCycle(model, ixs, r"C:\Users\adchu\Desktop\1.png", cycle)
# vcClient('127.0.0.1', '12000', toMaxArray, dT)

while True:
    cycle += 1
    toMaxArray = mainCycle(model, ixs, r"C:\Users\adchu\Desktop\1.png", cycle)
#     dT = setDelayTime('127.0.0.1', 12002)
    vcClient('127.0.0.1', '12000', toMaxArray, dT)

0
1
2
3
4
5


In [44]:
# load the model
model = VGG16()

# redefine model to output right after the first hidden layer
ixs = [2, 5, 9, 13, 17]
outputs = [model.layers[i].output for i in ixs]
model = Model(inputs=model.inputs, outputs=outputs)

# load the image with the required shape
img = load_img(r"C:\Users\adchu\Desktop\0.png", target_size=(224, 224))

# convert the image to an array
img = img_to_array(img)

# expand dimensions so that it represents a single 'sample'
img = expand_dims(img, axis=0)
# prepare the image (e.g. scale pixel values for the vgg)
img = preprocess_input(img)

# get feature map for first hidden layer
feature_maps = model.predict(img)
                                        
toMax = []
                                        
for i in range(len(feature_maps)):
    toMax.append(feature_maps[i][0].mean(2).flatten().tolist())


toMaxArray = []
#print(toMaxArray)
cycle = 0
print(cycle)

delay = 0.5

func = [3, 2, 1, 1, 1]

for i in range (len(toMax)):
    n = feature_maps[i][0].shape[0]
    pool = keras.layers.MaxPooling2D(pool_size=(func[i], func[i]))
    b = K.constant(feature_maps[i][0].mean(2).reshape(-1, n, n, 1))
    out = K.eval(pool(b))
    out = (out - out.min()) / (out.max() - out.min())
    #print(out.shape)
    toMaxArray += [out.flatten().tolist()]

vc = VVVVClient('127.0.0.1', int('12000'))
for i in range(len(toMaxArray)):
    vc.send_msg('/conv{}'.format(i), toMaxArray[i])
    vc.send_msg('/trig0', str(1))
    vc.send_msg('/trig0', str(0))
time.sleep(delay)

while True:
    cycle += 1
    # load the image with the required shape
    img = load_img(r"C:\Users\adchu\Desktop\1.png", target_size=(224, 224))

    # convert the image to an array
    img = img_to_array(img)

    # expand dimensions so that it represents a single 'sample'
    img = expand_dims(img, axis=0)
    # prepare the image (e.g. scale pixel values for the vgg)
    img = preprocess_input(img)

    # get feature map for first hidden layer
    feature_maps = model.predict(img)


    toMax = []

    for i in range(len(feature_maps)):
        toMax.append(feature_maps[i][0].mean(2).flatten().tolist())

    #print(toMax[4])

    toMaxArray = []
    #print(toMaxArray)
    print(cycle)
    
    func = [3, 2, 1, 1, 1]

    for i in range (len(toMax)):
        n = feature_maps[i][0].shape[0]
        pool = keras.layers.MaxPooling2D(pool_size=(func[i], func[i]))
        b = K.constant(feature_maps[i][0].mean(2).reshape(-1, n, n, 1))
        out = K.eval(pool(b))
        out = (out - out.min()) / (out.max() - out.min())
        #print(out.shape)
        toMaxArray += [out.flatten().tolist()]

    vc = VVVVClient('127.0.0.1', int('12000'))
    for i in range(len(toMaxArray)):
        vc.send_msg('/conv{}'.format(i), toMaxArray[i])
        vc.send_msg('/trig1', str(1))
        vc.send_msg('/trig1', str(0))
    time.sleep(delay)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31


KeyboardInterrupt: 

In [ ]:
# load the model
model = VGG16()

# redefine model to output right after the first hidden layer
ixs = [2, 5, 9, 13, 17]
outputs = [model.layers[i].output for i in ixs]
model = Model(inputs=model.inputs, outputs=outputs)

# load the image with the required shape
img = load_img(r"C:\Users\adchu\Desktop\0.png", target_size=(224, 224))

# convert the image to an array
img = img_to_array(img)

# expand dimensions so that it represents a single 'sample'
img = expand_dims(img, axis=0)
# prepare the image (e.g. scale pixel values for the vgg)
img = preprocess_input(img)

# get feature map for first hidden layer
feature_maps = model.predict(img)
                                        
toMax = []
                                        
for i in range(len(feature_maps)):
    toMax.append(feature_maps[i][0].mean(2).flatten().tolist())


toMaxArray = []
#print(toMaxArray)
cycle = 0
print(cycle)

delay = 0.5

func = [3, 2, 1, 1, 1]

for i in range (len(toMax)):
    n = feature_maps[i][0].shape[0]
    pool = keras.layers.MaxPooling2D(pool_size=(func[i], func[i]))
    b = K.constant(feature_maps[i][0].mean(2).reshape(-1, n, n, 1))
    out = K.eval(pool(b))
    out = (out - out.min()) / (out.max() - out.min())
    #print(out.shape)
    toMaxArray += [out.flatten().tolist()]

vc = VVVVClient('127.0.0.1', int('12000'))
for i in range(len(toMaxArray)):
    vc.send_msg('/conv{}'.format(i), toMaxArray[i])
    vc.send_msg('/trig0', str(1))
    vc.send_msg('/trig0', str(0))
time.sleep(delay)

while True:
    cycle += 1
    # load the image with the required shape
    img = load_img(r"C:\Users\adchu\Desktop\1.png", target_size=(224, 224))

    # convert the image to an array
    img = img_to_array(img)

    # expand dimensions so that it represents a single 'sample'
    img = expand_dims(img, axis=0)
    # prepare the image (e.g. scale pixel values for the vgg)
    img = preprocess_input(img)

    # get feature map for first hidden layer
    feature_maps = model.predict(img)


    toMax = []

    for i in range(len(feature_maps)):
        toMax.append(feature_maps[i][0].mean(2).flatten().tolist())

    #print(toMax[4])

    toMaxArray = []
    #print(toMaxArray)
    print(cycle)
    
    func = [3, 2, 1, 1, 1]

    for i in range (len(toMax)):
        n = feature_maps[i][0].shape[0]
        pool = keras.layers.MaxPooling2D(pool_size=(func[i], func[i]))
        b = K.constant(feature_maps[i][0].mean(2).reshape(-1, n, n, 1))
        out = K.eval(pool(b))
        out = (out - out.min()) / (out.max() - out.min())
        #print(out.shape)
        toMaxArray += [out.flatten().tolist()]

    vc = VVVVClient('127.0.0.1', int('12000'))
    for i in range(len(toMaxArray)):
        vc.send_msg('/conv{}'.format(i), toMaxArray[i])
        vc.send_msg('/trig1', str(1))
        vc.send_msg('/trig1', str(0))
    time.sleep(delay)

In [1]:
import sys
import socket
from time import sleep

s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.connect(('127.0.0.1',12001))
s.settimeout(2)

while True:
    try:
        msg = s.recv(4096)
    except socket.timeout as e:
        err = e.args[0]
        # this next if/else is a bit redundant, but illustrates how the
        # timeout exception is setup
        if err == 'timed out':
            sleep(1)
            print ('recv timed out, retry later')
            continue
        else:
            print (e)
            sys.exit(1)
    except socket.error as e:
        # Something else happened, handle error, exit, etc.
        print (e)
        sys.exit(1)
    else:
        if len(msg) == 0:
            print ('orderly shutdown on server end')
            sys.exit(0)
        else:
            # got a message do something :)
            print(print)

ConnectionRefusedError: [WinError 10061] No connection could be made because the target machine actively refused it

In [3]:
#looped transfer without break from vvvv to python

import socket
from vvvv import VVVVClient
import time

delayTime = 0

while True:
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    buffer_size = 8192
    s.bind(('127.0.0.1', 12002))
    s.setblocking(False)
    s.listen(5)
    (client, (ip,port)) = s.accept()
    s.setblocking(False)
    #print(client, port)
    data = client.recv(buffer_size)
    delayTime = data.decode()
    dT = float(delayTime.replace('\n', '').replace(';', ''))
    print(dT)
    s.close()
    vc = VVVVClient('127.0.0.1', int('16161'))
    vc.send_msg('/fb', str(1))
    vc.send_msg('/fb', str(0))
    time.sleep(.1)

OSError: [WinError 10048] Only one usage of each socket address (protocol/network address/port) is normally permitted

In [1]:
import socket
from vvvv import VVVVClient
import time

delayTime = 0

# while True:
print('while while')
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
buffer_size = 8192
s.bind(('127.0.0.1', 12002))
s.listen(1)
s.settimeout(5)
(client, (ip,port)) = s.accept()


while True:
    try:
        data = client.recv(buffer_size)
    except s.timeout as eto:
        break
        err = eto.args[0]
        # this next if/else is a bit redundant, but illustrates how the
        # timeout exception is setup
        if err == 'timeout: timed out':
            print(err)
#         else:
#             print (e+2222222)
    except socket.error as e:
        # Something else happened, handle error, exit, etc.
        print (e)
    else:
        if len(data) == 0:
            print('no msg')
            break
        else:
            # got a message do something :)
            delayTime = data.decode()
            dT = float(delayTime.replace('\n', '').replace(';', ''))
            print(dT)
#             s.close()
            vc = VVVVClient('127.0.0.1', int('16161'))
            vc.send_msg('/fb', str(1))
            time.sleep(.1)
            vc.send_msg('/fb', str(0))

while while


timeout: timed out

In [4]:
import socket
from vvvv import VVVVClient
import time
import errno

delayTime = 0

s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
buffer_size = 8192
s.bind(('127.0.0.1', 12002))
#print(client, port)

In [ ]:
import socket
from vvvv import VVVVClient
import time
import errno

while True:
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    buffer_size = 8192
    s.bind(('127.0.0.1', 12002))
    #print(client, port)
    s.listen(1)
    (client, (ip,port)) = s.accept()
    s.settimeout(1)
    
    while True:
        try:
            data = client.recv(buffer_size)
            delayTime = 0
            dT = 0
        except s.timeout as e:
            err = e.args[0]
            # this next if/else is a bit redundant, but illustrates how the
            # timeout exception is setup
            if err == 'timed out':
                print('timed out err' + e)
                break
            else:
                print (e)
                break
        except s.error as e:
            # Something else happened, handle error, exit, etc.
            print ('Something else happened, handle error, exit, etc' + e)
            break
        else:
            if len(data) == 0:
                print(dT)
                break
            else:
                # got a message do something :)
                delayTime = data.decode()
                dT = float(delayTime.replace('\n', '').replace(';', ''))
                print(dT)
                vc = VVVVClient('127.0.0.1', int('16161'))
                vc.send_msg('/fb', str(1))
                time.sleep(.2)
                vc.send_msg('/fb', str(0))
                break
    s.close()
    break

In [ ]:
import socket
from vvvv import VVVVClient
import time
import errno

def setDelayTime(ip, port):
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    buffer_size = 8192
    s.bind((ip, port))
    #print(client, port)
    s.listen(1)
    (client, (ip,port)) = s.accept()
    s.settimeout(1)

    while True:
        data = client.recv(buffer_size)
        delayTime = 0
        dT = 0

        if len(data) == 0:
#                 print(dT)
            break
        else:
            # got a message do something :)
            delayTime = data.decode()
            dT = float(delayTime.replace('\n', '').replace(';', ''))
            print(dT)
            vc = VVVVClient('127.0.0.1', int('16161'))
            vc.send_msg('/fb', str(1))
            time.sleep(.2)
            vc.send_msg('/fb', str(0))
        s.close()
        return dT
    
while True:
    setDelayTime('127.0.0.1', 12002)

39.09
39.11
39.07
39.03
39.0
38.98
